In [3]:
!pip install pyomo
!pip install cplex -q
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

In [6]:
model = pyo.ConcreteModel()

model.i = pyo.Set(initialize = ['City1','City2','City3','City4','City5'])
model.j = pyo.Set(initialize = ['City1','City2','City3','City4','City5'])

model.ii = pyo.Set(initialize = ['City2','City3','City4','City5'])

# defining parameters using pandas to load data
C = pd.read_excel('S5P3_Data.xlsx',sheet_name='TSP', header=0, index_col=0, usecols = 'A:F', nrows = 5)

model.x = pyo.Var(model.i, model.j, domain = pyo.Binary)
x = model.x
model.u = pyo.Var(model.i, domain = pyo.NonNegativeReals)
u = model.u

def objective_rule(model):
  return sum(sum(C[i][j] * x[i,j] for i in model.i) for j in model.j)
model.obj = pyo.Objective(rule = objective_rule)

def con1(model,j):
  return sum(x[i,j] for i in model.i) == 1
model.con1  = pyo.Constraint(model.i, rule = con1)

def con2(model,i):
  return sum(x[i,j] for j in model.j) == 1
model.con2  = pyo.Constraint(model.j, rule = con2)

def con3(model,i,j):
  if i != j :
    return u[i] - u[j] + 5 * x[i,j] <= 4
  else:
    return u[i] - u[i] == 0
model.con3 = pyo.Constraint(model.ii,model.ii, rule = con3)

opt = SolverFactory('cplex_direct')
results = opt.solve(model)

print(results)
print("Objective function", model.obj())
for i in model.i:
  for j in model.j:
    print('salesman goes from',i,'to the',j,'=',x[i,j]())




Problem: 
- Name: 
  Lower bound: 668.0
  Upper bound: 668.0
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 30
  Number of binary variables: 25
  Number of integer variables: 0
  Number of continuous variables: 5
  Number of nonzeros: None
  Sense: 1
Solver: 
- Name: CPLEX 22.1.1.0
  Status: ok
  Wallclock time: 0.0058367252349853516
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function 668.0
salesman goes from City1 to the City1 = -0.0
salesman goes from City1 to the City2 = 0.0
salesman goes from City1 to the City3 = 1.0
salesman goes from City1 to the City4 = -0.0
salesman goes from City1 to the City5 = 0.0
salesman goes from City2 to the City1 = -0.0
salesman goes from City2 to the City2 = -0.0
salesman goes from City2 to the City3 = -0.0
salesman goes from City2 to the City4 = 0.0
salesman goes from City2 to the City5 = 1.0
salesman goes from City3 to the City1 = 0.0
salesman goes 